In [ ]:
import hdb
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import os

## 流程说明

1. 获取每天的tick数据
    
    ``db = hdb.DB(r"Z:\hdb_data")``
    ``file = db.open_file(r"marketdata\tick_yyyymmdd", mode="r")``
    
    注意需要file.close()

2. 获取每天的沪深300成分股

    ``file.get_codelist(cl_name = "HS300").symbols``

3. 按照成分股的代获取当天所有成分股的tick数据

    ``read_data(f, symbols, data_type, begin=None, end=None)``

4. 简单清洗数据

    ``data_process(data, select_var)``

5. 计算每个tick的数据

    固定交易成本50万
    
    ``each_tick(data, ask_bid)``

6. 使用groupby按照股票分类计算每天tick的冲击成本的平均数

    ``data.groupby("stock_code")["ask"].mean()``

7. 使用流通市值加权

    7.1 获取自由流通股数量
    
    7.2 计算流通市值
        
        前一天收盘价 乘 自由流通股数量

In [ ]:
# obtain trading date
date = pd.read_csv("Code_date.csv")
date = date[["Date"]]
date = date["Date"].unique().tolist()
date = {"Date":date}
date = pd.DataFrame(date)
date["Date"] = pd.to_datetime(date["Date"])
date

In [ ]:
date = pd.read_excel("trading_date2023.xlsx")
date["Date"] = pd.to_datetime(date["Date"])
date

In [ ]:
date["date_code"] = date["Date"].map(lambda x: x.strftime("%Y%m%d"))
date

In [ ]:
#将上述过程封装后可以调用read_data
def read_data(f, symbols, data_type, begin=None, end=None):
    if (begin is None) and (end is None):
        task = f.open_read_task(symbols=symbols, types=[data_type])
    elif begin is None:
        task = f.open_read_task(end_time=end, symbols=symbols, types=[data_type])
    elif end is None:
        task = f.open_read_task(begin_time=begin, symbols=symbols, types=[data_type])
    else:
        task = f.open_read_task(begin_time=begin, end_time=end, symbols=symbols, types=[data_type])
    items = task.read()
    data_type = f.data_types[items['type_id'][0]]
    data = data_type.items_data(items['data'])
    df = pd.DataFrame()
    for name in data_type.dtype.fields:
        field_type = data.dtype.fields[name][0]
        if 0 == field_type.ndim:
            df[name] = data[name]
        elif 1 == field_type.ndim:
            for idx in range(field_type.shape[0]):
                df[name + '.' + str(idx)] = data[name][:, idx]

    df['symbol'] = items['symbol']
    df['date'] = items['trading_day']
    
    return df

In [ ]:
var = ['time',  'open', "pre_close",'high',  "low",'match',
       'ask_price.0', 'ask_price.1', 'ask_price.2', 'ask_price.3',
       'ask_price.4', 'ask_price.5', 'ask_price.6', 'ask_price.7',
       'ask_price.8', 'ask_price.9', 'ask_vol.0', 'ask_vol.1', 'ask_vol.2',
       'ask_vol.3', 'ask_vol.4', 'ask_vol.5', 'ask_vol.6', 'ask_vol.7',
       'ask_vol.8', 'ask_vol.9', 'bid_price.0', 'bid_price.1', 'bid_price.2',
       'bid_price.3', 'bid_price.4', 'bid_price.5', 'bid_price.6',
       'bid_price.7', 'bid_price.8', 'bid_price.9', 'bid_vol.0', 'bid_vol.1',
       'bid_vol.2', 'bid_vol.3', 'bid_vol.4', 'bid_vol.5', 'bid_vol.6',
       'bid_vol.7', 'bid_vol.8', 'bid_vol.9', 'num_trades', 'volume',
       'turnover', 'total_bid_vol', 'total_ask_vol', 'high_limited',
        'sd2', 'trading_code',
         'date', 'stock_code']
def data_process(data, select_var = var):
    # decode
    data['stock_code'] = data['symbol'].map(lambda x: x.decode("utf-8"))
    data['trading_code'] = data['trading_phase_code'].map(lambda x: x.decode("utf-8"))
    # select variabel
    output = data[var]
    output = output[output["trading_code"].isin(["T111", 'T0      '])]
    # process some error value
    output = output.query('open != 0')
    output = output[output["ask_price.0"] != 0]
    output = output[output["bid_price.0"] != 0]
    output["high_limit"] = output["pre_close"] * 1.098
    output["low_limit"] = output["pre_close"] * 0.92
    
    return output

In [ ]:
def each_tick_ask(row, ask_bid = "ask"):
    
    if row["ask_price.0"] >= row["high_limit"]:
        cost_point = 10
    
    else:
        inital_price = ask_bid + "_price.0"
        total_money = np.array(row["cost"])

        for i in range(0,10):
            price_col = ask_bid + "_price." + str(i)
            quantity_col = ask_bid + "_vol." + str(i)
            price = np.array(row[price_col]) / 10000
            quantity = np.array(row[quantity_col])
            
            if total_money > price and price != 0:
                total_money -= price * min(int(quantity), int(total_money // price))
                max_price = price
                
                continue
            else:
                break
            
        cost_point = (max_price / (np.array(row[inital_price])/10000) - 1)*100

    return cost_point


def each_tick_bid(row, ask_bid = "bid"):
    
    if row["bid_price.0"]<= row["low_limit"]:
        cost_point = -10
    
    else:
        inital_price = ask_bid + "_price.0"
        total_money = np.array(row["cost"])
        for i in range(0,10):
            price_col = ask_bid + "_price." + str(i)
            quantity_col = ask_bid + "_vol." + str(i)
            price = np.array(row[price_col]) / 10000
            quantity = np.array(row[quantity_col])

            if total_money > price and price != 0:
                total_money -= price * min(int(quantity), int(total_money // price))
                max_price = price
                continue
            else:
                break

        cost_point = (max_price / (np.array(row[inital_price])/10000) - 1)*100

    return -cost_point



In [ ]:
# calculate impact cost
def impact_cost(data):
    
    daily_ask = data.groupby(["stock_code", "pre_close"])["ask"].mean()
    daily_bid = data.groupby(["stock_code", "pre_close"])["bid"].mean()
    
    output = pd.concat([daily_ask, daily_bid], axis = 1)
    output = output.reset_index()
    output["pre_close"] = output["pre_close"]/ 10000
    
    return output

In [ ]:
def find_captial(f, stock_index):
    location = f.codetable.symbols
    
    location_index = []
    for i in stock_index:
        if i in location:
            location_index.append(location.index(i))
    
    codetable_pd = pd.DataFrame(f.ci_type.items_data(file.codetable.data))
    captial_list = codetable_pd.loc[location_index,"capital"]
    captial = {"stock_code":stock_index, "capital": captial_list}
    captial = pd.DataFrame(captial)
    
    return captial

## 循环

In [ ]:
date_2017 = date[date["Date"] >= "2023-04-01"]
date_2017 = date_2017[date_2017["Date"] < "2023-07-01"]
date_2017 = date_2017["date_code"]
date_2017

In [ ]:
ask_list = []
bid_list = []

In [ ]:
for i in date_2017:
    db = hdb.DB(r"Z:\hdb_data")
    temp_file = str("marketdata/tick_") + i
    file = db.open_file(temp_file, mode="r")
    print("Star", i)
    index_comp_list = file.get_codelist(cl_name = "HS300").symbols
    
    data = read_data(file, 
                 begin = datetime.datetime(int(i[0:4]) , int(i[4:6]),int(i[6:8]), 9, 30), 
                 end = datetime.datetime(int(i[0:4]), int(i[4:6]), int(i[6:8]), 14, 57), # 集合竞价开始于14：57 
                 symbols = index_comp_list, 
                 data_type = "SecurityTick")
    captial = find_captial(f = file, stock_index = index_comp_list)
    file.close()
    
    data = data_process(data = data)
    data["cost"] = 500000.00
    
    data["ask"] = data.apply(each_tick_ask,axis = 1)
    data["bid"] = data.apply(each_tick_bid,axis = 1)
    
    data_daily = impact_cost(data = data)
    
    stock_captial = data_daily.merge(captial, on = "stock_code")
    stock_captial["free_market"] = (stock_captial["pre_close"]/10000)*stock_captial["capital"]
    ask = np.average(stock_captial["ask"], weights = stock_captial["free_market"])
    bid = np.average(stock_captial["bid"], weights = stock_captial["free_market"])
    ask_list.append(ask)
    bid_list.append(bid) 
    print("complet", i)

In [ ]:
len(ask_list)

In [ ]:
date_2017 = date[date["Date"] >= "2021-06-01"]
date_2017 = date_2017[date_2017["Date"] < "2022-01-01"]
date_2017 = date_2017["Date"]
date_2017

In [ ]:
df = {"Date":date_2017, "Ask": ask_list, "Bid": bid_list}
df = pd.DataFrame(df)
df = df.set_index("Date")
df